In [1]:
# torch is there, torchvision is there
!pip install timm

import os
import numpy as np

# Feature extraction
from torch.utils.data import DataLoader
import torch
from torchvision import transforms
import torchvision
import timm

# No grad's needed
torch.set_grad_enabled(False)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

from google.colab import drive

mount_str = "/content/drive" # mount gdrive
drive.mount(mount_str)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [15]:
# Dataset
#dataset_root_path = f"{mount_str}/MyDrive/microcosmus/rotifer-locomotion/data/images/resized_cropped_squarebbox_gs_224/"
dataset_root_path = f"{mount_str}/MyDrive/microcosmus/rotifer-locomotion/data/images/resized_cropped_squarebbox_rgb_wbg_224/"
ds = timm.data.create_dataset("folder", root=dataset_root_path)  # ds = torchvision.datasets.ImageFolder(root=dataset_root_path)
print(ds)

# Transforms
tfs = transforms.Compose([
    #transforms.Resize(img_size),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.5], std=[0.5]),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Add transforms
ds.transform = tfs
dataloader = DataLoader(ds, batch_size=96, num_workers=12, shuffle=False)

# possible timm transforms instead of pytorch
#data_config = timm.data.resolve_model_data_config(model)
#transforms = timm.data.create_transform(**data_config, is_training=False)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [25]:
# Model selection
model_names = ["eva02_base_patch14_224.mim_in22k",
               "hiera_base_plus_224.mae_in1k_ft_in1k",
               "vit_base_patch16_224_in21k"]  # "vit_base_patch16_rope_reg1_gap_256.sbb_in1k"

for model_name in model_names:
    
    # Model
    model_params = {"model_name" : model_name, "pretrained" : True,
                    "num_classes" : 0, "global_pool" : "avg"}  # instead of num_classes, set features_only=True
    
    model = timm.create_model(**model_params).eval();
    model.to(device);

    # FEATURE Extraction
    features = list()
    for nth, (images, labels) in enumerate(dataloader):
        images, labels = images.to(device), labels.to(device)
        with torch.no_grad():
            outputs = model(images)
        print(nth)
        features.append(outputs.detach().cpu().numpy())

    feature_vectors = np.concatenate(features)
    del features
    print(feature_vectors.shape)

    # SAVE FEATURES
    SAVE_FEATS = True
    save_features_path = f"{mount_str}/MyDrive/microcosmus/rotifer-locomotion/data/results/feature_vectors_{model_params['model_name']}.npy"

    if SAVE_FEATS:
        np.save(save_features_path, feature_vectors)
    else:
        feature_vectors = np.load(save_features_path)

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

AssertionError: Input height (224) doesn't match model (256).